# Extract data from doc and store in table

A company use all sort of information for decision making, these information can be anywhere, document, e-mail, or even powerpoint. Here is a tutorial on how to extract data from data stored in document and transform into a table format.

Currently, there is lack of library that convert .doc file into text but there are several work for docx. So, the first process is to convert these doc file into docx, program like LibreOffice will be idea of this task especially it can be install in an AWS server and run in command line. Imagine there are thousands of documents, we want to write the code and let the machine run it.

In [1]:
import os
from subprocess import call
files = os.listdir("doc")
files

['Editable Credit Report.doc',
 '.DS_Store',
 'SAMPLE_FORMAT_FOR_A_FINANCIAL_REPORT.doc',
 'Form for Review Comments.doc',
 'FSChimimport interim 06008 BG v3(2)_ENG.doc',
 'mastersinstructions.doc']

In [16]:
!ls /Applications/LibreOffice.app/Contents/MacOS/

gengal       regview      soffice      uno          unopkg       uri-encode
regmerge     senddoc      ui-previewer unoinfo      urelibs      xpdfimport


In [27]:
for f in files:
    file_path = 'doc/' + f
    call(['/Applications/LibreOffice.app/Contents/MacOS/soffice', '--headless', '--convert-to', 'docx', file_path, '--outdir', 'doc'])

In [32]:
files = os.listdir("doc")
files = [f for f in files if 'docx' in f]
files = [f for f in files if not f.startswith('.')]
files

['Form for Review Comments.docx',
 'FSChimimport interim 06008 BG v3(2)_ENG.docx',
 'mastersinstructions.docx',
 'SAMPLE_FORMAT_FOR_A_FINANCIAL_REPORT.docx',
 'Editable Credit Report.docx']

In [33]:
import docx2txt
import pandas as pd
import numpy as np

In [34]:
def fn_docx2txt(path, filename):
    # Using python-docx library to parse document into string list, filename and path
    doc = docx2txt.process(path + filename)
    doc = doc.encode("utf-8")
    text = doc.split('\n\n')
    profile = {}
    profile['path'] = path
    profile['filename'] = filename
    return text, profile   

## Extract data from a simple table

In [59]:
text, profile = fn_docx2txt('doc/', files[0])
profile

{'filename': 'Form for Review Comments.docx', 'path': 'doc/'}

![](doc/p3_Form for Review Comments.png)

In [53]:
text[7:30]

['Document Review \xe2\x80\x93 Comment Sheet',
 'Comment Sheet Information (to be filled in by the Reviewer)',
 'Comment Sheet Reference\xc2\xa0:',
 '<Form for Review Comments \xe2\x80\x93 Version - COMT_XY>',
 'Comment Sheet Date\xc2\xa0:',
 '<dd/mm/yyyy>',
 'Document Information (to be filled in by the Author)',
 'Document Title\xc2\xa0:',
 'Accompanying report to the recommendation of the European Railway Agency ',
 'Document Reference\xc2\xa0:',
 'ERA-REC-123-ACR',
 'Document Version\xc2\xa0:',
 '0.5',
 'Document Date\xc2\xa0:',
 '05/06/2015',
 'Date By Which Document Reviewed\xc2\xa0:',
 '15/09/2015',
 'Document Reviewed by (to be filled in by the Reviewer)',
 'Organisation or Company\xc2\xa0:',
 '',
 'Name\xc2\xa0:',
 '',
 'E-mail\xc2\xa0:']

### Function to extract fields

Basically, we find the field name e.g. Document Title, then the next line will be the field content.

In [54]:
def title(x):
    keywords = ['Document Title']
    for line in range(len(x)):
        if any(w in x[line] for w in keywords):
            FIELD = x[line+1]
            break
        else:
            FIELD = np.NaN
    return FIELD

def reference(x):
    keywords = ['Document Reference']
    for line in range(len(x)):
        if any(w in x[line] for w in keywords):
            FIELD = x[line+1]
            break
        else:
            FIELD = np.NaN
    return FIELD

def ddate(x):
    keywords = ['Document Date']
    for line in range(len(x)):
        if any(w in x[line] for w in keywords):
            FIELD = x[line+1]
            break
        else:
            FIELD = np.NaN
    return FIELD

def rdate(x):
    keywords = ['Document Reviewed']
    for line in range(len(x)):
        if any(w in x[line] for w in keywords):
            FIELD = x[line+1]
            break
        else:
            FIELD = np.NaN
    return FIELD

info_dict = {}
info_dict['TEXT'] = title(text)
info_dict['REF'] = reference(text)
info_dict['D_DATE'] = ddate(text)
info_dict['R_DATE'] = rdate(text)

info_dict

{'D_DATE': '05/06/2015',
 'REF': 'ERA-REC-123-ACR',
 'R_DATE': '15/09/2015',
 'TEXT': 'Accompanying report to the recommendation of the European Railway Agency '}

In [55]:
pd.DataFrame([info_dict]).T

,0
D_DATE,05/06/2015
REF,ERA-REC-123-ACR
R_DATE,15/09/2015
TEXT,Accompanying report to the recommendation of t...


## Extract data from a financial statement

In [61]:
text, profile = fn_docx2txt('doc/', files[1])
profile

{'filename': 'FSChimimport interim 06008 BG v3(2)_ENG.docx', 'path': 'doc/'}

![](doc/p2.png)

### Create a function that extract the paragraph and remove the empty

In [78]:
def extract_paragraph(text, keyword1, keyword2, remove_empty = False):
    paragraph = []
    for line in range(len(text)):
        if any(w in text[line] for w in keyword1):
            paragraph.append(text[line])
            nl = 1
            while any(w in text[line+nl] for w in keyword2) ==  False:
                if (line + nl + 1) >= len(text):
                    paragraph.append(text[line+nl])
                    break
                else:
                    paragraph.append(text[line+nl])
                    nl += 1
            break
    paragraph.append('THE END')
    if remove_empty == True:
        paragraph = [line for line in paragraph if line is not '']
    return paragraph

In [79]:
keyword1 = ['Notes to the Interim Financial Statements ']
keyword2 = ['Prepared by:']
text2 = extract_paragraph(text, keyword1, keyword2, remove_empty = True)
text2[:40]

['Notes to the Interim Financial Statements ',
 '9',
 'Interim Balance Sheet',
 'Notes',
 '30 June',
 '31 December',
 '30 June',
 '2008',
 '2007',
 '2007',
 'BGN\xe2\x80\x99000',
 'BGN\xe2\x80\x99000',
 'BGN\xe2\x80\x99000',
 'Assets',
 'Non-current assets ',
 'Property, plant and equipment ',
 '4',
 '         28 964 ',
 '21 838',
 '19 101',
 'Investment property',
 '           1 130 ',
 '1 170',
 '1 214',
 'Investments in subsidiaries ',
 '       388 693 ',
 '340 387',
 '185 909',
 'Investments in associates ',
 '         45 670 ',
 '18 767',
 '18 052',
 'Intangible assets',
 '5',
 '              123 ',
 '247',
 '371',
 'Long-term financial assets',
 '           1 245 ',
 '19 510']

### First, find the financial date

In [85]:
def isdate(x):
    if x == '':
        return False
    else:        
        try:
            pd.to_datetime(x)
            return True
        except:
            return False
        
def financial_date(x):
    keywords = ['Notes']
    F_date = []
    for line in range(len(x)-1):
        if any(w in x[line] for w in keywords):
            nl = 1
            while nl < 10:
                date = x[line+nl]
                year = x[line+nl+3]
                if isdate(date + year):
                    F_date.append(pd.to_datetime(date + year).date())
                    nl += 1
                else:
                    nl += 1
            break
    return F_date

Fin_info = {}
Fin_info['F_date'] = financial_date(text2)
Fin_info['F_date']

[datetime.date(2008, 6, 30),
 datetime.date(2007, 12, 31),
 datetime.date(2007, 6, 30)]

In [127]:
def isfloat(x):
    x = x.replace(',','')
    try:
        float(x)
        return True
    except:
        return False

def investment_property(x):
        keywords = ['Investment property']
        FIELDS = []
        for line in range(len(x)):
            if any(w in x[line] for w in keywords):
                nl = 1
                while isfloat(x[line+nl].replace(' ', '')):
                    if (line + nl + 1) >= len(x):
                        break
                    else:
                            try:
                                FIELDS.append(float(x[line+nl].replace(' ','')))
                            except:
                                FIELDS.append(x[line+nl].replace(' ', ''))
                            nl += 1
                if len(FIELDS) == NO_OF_YEARS:
                    break
        # FIELDS = [i for i in FIELDS if type(i) == float]
        if len(FIELDS) > NO_OF_YEARS:
            FIELDS = FIELDS[-1*(NO_OF_YEARS):]
        return FIELDS
    
# if there are some unwanted fields
def property_plant_equipment(x):
        keywords = ['Property, plant and equipment']
        FIELDS = []
        for line in range(len(x)):
            if any(w in x[line] for w in keywords):
                nl = 1
                while isfloat(x[line+nl].replace(' ', '')):
                    if (line + nl + 1) >= len(x):
                        break
                    else:
                            try:
                                FIELDS.append(float(x[line+nl].replace(' ','')))
                            except:
                                FIELDS.append(x[line+nl].replace(' ', ''))
                            nl += 1
                if len(FIELDS) == NO_OF_YEARS:
                    break
        # FIELDS = [i for i in FIELDS if type(i) == float]
        if len(FIELDS) > NO_OF_YEARS:
            FIELDS = FIELDS[-1*(NO_OF_YEARS):]
        return FIELDS    
    
# if there are some fields that is not number but we still want it
def longterm_receivables(x):
        keywords = ['Long-term receivables due from ']
        FIELDS = []
        for line in range(len(x)):
            if any(w in x[line] for w in keywords):
                nl = 1
                while isfloat(x[line+nl].replace(' ', '')) or (x[line+nl] == '-'):
                    if (line + nl + 1) >= len(x):
                        break
                    else:
                            try:
                                FIELDS.append(float(x[line+nl].replace(' ','')))
                            except:
                                FIELDS.append(x[line+nl].replace(' ', ''))
                            nl += 1
                if len(FIELDS) == NO_OF_YEARS:
                    break
        # FIELDS = [i for i in FIELDS if type(i) == float]
        if len(FIELDS) > NO_OF_YEARS:
            FIELDS = FIELDS[-1*(NO_OF_YEARS):]
        return FIELDS



In [132]:
NO_OF_YEARS = len(Fin_info['F_date'])
Fin_info['property_plant_equipment'] = property_plant_equipment(text2)
Fin_info['investment_property'] = investment_property(text2)
Fin_info['longterm_receivables'] = longterm_receivables(text2)
Fin_info

{'F_date': [datetime.date(2008, 6, 30),
  datetime.date(2007, 12, 31),
  datetime.date(2007, 6, 30)],
 'investment_property': [1130.0, 1170.0, 1214.0],
 'longterm_receivables': [81052.0, 72465.0, '-'],
 'property_plant_equipment': [28964.0, 21838.0, 19101.0]}

### Create a dictionary with each date as a field

In [143]:
var = Fin_info.keys()
try:
    var.remove('F_date')
except:
    pass

Fin_info_DICT = {}
Fin_info_DICT['F_date'] = Fin_info['F_date']
try:
    for y in range(len(Fin_info['F_date'])):
        for v in var:
            if Fin_info[v][y] == '-':
                Fin_info_DICT[v + '_' + str(Fin_info['F_date'][y])] = np.NaN
            else:
                try:
                    Fin_info_DICT[v + '_' + str(Fin_info['F_date'][y])] = Fin_info[v][y]
                except:
                    Fin_info_DICT[v + '_' + str(Fin_info['F_date'][y])] = np.NaN
except:
    pass

In [146]:
Fin_info_DICT

{'F_date': [datetime.date(2008, 6, 30),
  datetime.date(2007, 12, 31),
  datetime.date(2007, 6, 30)],
 'investment_property_2007-06-30': 1214.0,
 'investment_property_2007-12-31': 1170.0,
 'investment_property_2008-06-30': 1130.0,
 'longterm_receivables_2007-06-30': nan,
 'longterm_receivables_2007-12-31': 72465.0,
 'longterm_receivables_2008-06-30': 81052.0,
 'property_plant_equipment_2007-06-30': 19101.0,
 'property_plant_equipment_2007-12-31': 21838.0,
 'property_plant_equipment_2008-06-30': 28964.0}

### Return the dictionary into table

In [149]:
def dict2table(x):
    # make dictionary into table format
    df = pd.DataFrame([x])
    df_all = pd.DataFrame()
    col = []
    for y in x['F_date']:
        globals()[str(y)] = df[[k for k in x.keys() if (str(y) in k)]].copy()
        globals()[str(y)].rename(columns=lambda x: x.replace('_' + x.split('_')[-1], ''), inplace=True)
        globals()[str(y)] = globals()[str(y)].T[0].sort_index()
        col.append(str(y))
        df_all = pd.concat([df_all, pd.DataFrame({str(y):globals()[str(y)]})], axis = 1)
    df_all = df_all[col]
    return df_all

In [150]:
dict2table(Fin_info_DICT)

,2008-06-30,2007-12-31,2007-06-30
investment_property,1130.0,1170.0,1214.0
longterm_receivables,81052.0,72465.0,NaN
property_plant_equipment,28964.0,21838.0,19101.0
